# Data Generation for Plots Notebook

This notebook generates data used to contruct `binding peak` tables and `venn diagrams` for ChIP-pro, as well as other  visuals. But before you run the notebook, make sure of the following: 

> Make sure to rename TF in the gff file to match the TF convention and not gene convension.

In [30]:
import numpy as np 
import pandas as pd
import glob
import urllib                      
import gzip
from collections import defaultdict

pd.set_option('display.max_columns', 500)

# Functions

## Generate Binding peaks 

This function assigns bindings peaks for each TF and fins the corresponsing gene targets using the `gene_info.csv` table. 

In [2]:
def annotate_peaks(TF_name, TF_condition, peak_df,margin,gene_info):
    
    res_df = peak_df.copy()
#     TF = peak_df.condition_name[1][:4]
    for i,row in res_df.iterrows():
        pos = row['binding_peak_start']
        # Identify genes within MARGIN nt of binding peak
        close_genes = gene_info[(gene_info.start_codon_pos > pos-margin) 
                                & (gene_info.start_codon_pos < pos+margin)]
        for strand,group in close_genes.groupby('strand'):
            #Remove genes that are completely transcribed before binding peak
            if strand == '+':
                group = group[group.stop > pos]
            else:
                group = group[group.start < pos]

            operon = group.operon.unique()
            # Ensure that we're only identifying one operon on either side of binding peak
            if len(operon) > 1:
                print (operon)

            # Get all genes in operon
            bnums = gene_info[gene_info.operon.isin(operon)].index
            
            ## Add gene information to dataframe
            if strand == '+':
                res_df.loc[i,'TU_p'] = ','.join(operon)
                res_df.loc[i,'genes_p'] = ','.join(bnums)
            else:
                res_df.loc[i,'TU_m'] = ','.join(operon)
                res_df.loc[i,'genes_m'] = ','.join(bnums)
    res_df['index'] = [TF_name +'-' + str(i) for i in range(1,peak_df.shape[0]+1)]
    res_df['condition'] = [ TF_name.lower() + " + " + TF_condition for i in peak_df.condition_name]
#     [peak_df.condition_name[2][:4]+' + '+peak_df.condition_name[1][5:8]]*peak_df.shape[0]
    cols = ['index','condition','binding_peak_start','binding_peak_end',
            'binding_peak_strength','TU_p','genes_p','TU_m','genes_m']
    return res_df.reindex(columns = cols)

## Validate Binding Peaks 

this function validates the accuracy of every gene target from the already indetified gene list for every binding site, identified from the previous function  

In [68]:
def validate_peak_info(df,gene_info):
    locusTag = defaultdict(list)
    geneName = defaultdict(list)
    for i,row in df.iterrows():
        BP = row['binding_peak_start']
        idx_name = i
        genes = [row['genes_p'] , row['genes_m']]
        if ((genes[0] == '' or str(genes[0]) == 'nan') & (genes[1] == '' or str(genes[1]) == 'nan')):
            locusTag[idx_name].append('')
            geneName[idx_name].append('')
        for gene in genes:
            if gene == '' or str(gene) == 'nan':
                continue 
            gene_list = gene.split(',')
            for g in gene_list: 
                name = DF_gene_info.loc[g].gene_name 
                strand = DF_gene_info.loc[g].strand 
                start = DF_gene_info.loc[g].start
                stop = DF_gene_info.loc[g].stop
                if ((start > BP) & (stop > BP) & (strand == '+')) | ((start < BP) & (stop < BP) & (strand == '-')):
                        locusTag[idx_name].append(g)
                        geneName[idx_name].append(name)
                elif ((start < BP) & (stop > BP)) | ((start > BP) & (stop < BP)):
                    locusTag[idx_name].append(g)
                    geneName[idx_name].append(name)
                    
    for k, v in locusTag.items():
        if ((len(v) == 1) & (v[0] == '')): 
            locusTag[k] = ''
            continue
        genes = ','.join(locusTag[k])
        locusTag[k] = genes

    for k, v in geneName.items():
        if ((len(v) == 1) & (v[0] == '')): 
            geneName[k] = ''
            continue
        genes = ','.join(geneName[k])
        geneName[k] = genes

    df_complete = df.loc[:,['index','condition','binding_peak_start',
                            'binding_peak_end','binding_peak_strength']]
    df_complete['target_locus'] = locusTag.values()
    df_complete['target_genes'] = geneName.values()
    return df_complete

# Venn Diagrams

Make sure to update the list of `TF_names` as Ye adds more gff files into the dropbox

In [5]:
def Venn_data_gen(Peak_DF): 
    TF_name = Peak_DF['index'][1][:4]
    gene_list = [i for i in Peak_DF.target_genes if i != '']
    chip_data= ','.join(list(gene_list)).split(',')
    reg_data = TRN_data[TF_name]
    all_genes = [i  for i in chip_data if i not in reg_data] + reg_data
    
    
    TF = TF_name
    reg_genes=reg_data
    reg_only = []
    chip_genes=chip_data
    chip_only = []
    shared_genes=[]
    for i in all_genes: 
        if (i in reg_data) & (i not in chip_data):
            reg_only.append(i)
        elif (i in chip_data) & (i not in reg_data):
            chip_only.append(i)
        elif (i in chip_data) & (i in reg_data):
            shared_genes.append(i)
            
    values = [TF,
          len(reg_genes),
          len(reg_only),
          len(chip_genes),
          len(chip_only),
          len(shared_genes),
          len(all_genes)]
    
    index_name = ['TF',
              'reg_genes',
              'reg_only',
              'chip_genes',
              'chip_only',
              'shared_genes',
              'all_genes']
    
    genes = ['; '.join(precise2_TRN.source[precise2_TRN.index == TF_name].unique()),
         reg_genes,
         reg_only,
         chip_genes,
         chip_only,
         shared_genes,
         all_genes]
    
    same1 = defaultdict(list)
    for i in range(0,len(index_name)):
        same1[index_name[i]].append(values[i])

    finall = pd.DataFrame.from_dict(same1, orient='index', columns = ['value'])
    finall['list'] = genes
    for i, row in finall.iterrows(): 
        if row.value == 0:
            finall.list[i] = ''
    return finall

# Binding Width Histograms

In [7]:
def binding_width_gen(TF, peak_df,out_dir): 
    widths = pd.DataFrame(peak_df.binding_peak_end - peak_df.binding_peak_start + 1, columns=["binding_width"])
    widths.to_csv(out_dir+TF+'_widths.csv')

# Peak Position Scatter

In [8]:
def peak_position_gen(TF, final_annot_DF,out_dir): 
    peak_scatter_df = pd.DataFrame(columns=final_annot_DF.columns.to_list()+['gene'] +['normalized_dist'])
    counter=0

    for i in range(len(final_annot_DF)):
        peak = final_annot_DF.loc[i+1,:]
        peak_center = np.mean([peak.binding_peak_start, peak.binding_peak_end])

        for gene in peak.target_genes.split(","):
            gene_info = DF_gene_info[DF_gene_info.gene_name == gene]
            try:
                if gene_info.strand[0] == "-":
                    dist = float((gene_info.stop - peak_center)/gene_info.length)
                else:
                    dist = float((peak_center - gene_info.start)/gene_info.length)
                peak_scatter_df.loc[counter] = peak.to_list()+[gene, dist]
                counter+=1
            except:
                continue

    peak_scatter_df['binding_peak_center'] = peak_scatter_df.binding_peak_start/2 + peak_scatter_df.binding_peak_end/2
    peak_scatter_df.to_csv(out_dir+TF+'_positions.csv')

# E. coli

In [9]:
org_folder = "../data/e_coli/"

In [10]:
TF_list = pd.read_csv(org_folder+'TF_list.csv', index_col=0)

TF_list

,TF,Organism,Strain,Media,Supplement,genome_id,organism_id,num_binding_sites,num_samples,num_cond,project,binding tables,BWcond1_1_name,BWcond1_1,BWcond1_2_name,BWcond1_2,BWcond2_1_name,BWcond2_1,BWcond2_2_name,BWcond2_2,BWcond3_1_name,BWcond3_1,BWcond3_2_name,BWcond3_2,BWcond4_1_name,BWcond4_1,BWcond4_2_name,BWcond4_2
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,BaeR,Escherichia coli,K-12 MG1655,LB,EtOH,NC_000913_3,e_coli,NaN,4,1,TCS,EtOH|baer_EtOH_curated.gff,bio-rep1 – R1,BaeR_R1_S31_R1.bw,bio-rep1 – R2,BaeR_R1_S31_R2.bw,bio-rep2 – R1,BaerR_R2_S32_R1.bw,bio-rep2 – R2,BaerR_R2_S32_R2.bw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CpxR,Escherichia coli,K-12 MG1655,LB,EtOH,NC_000913_3,e_coli,NaN,2,1,TCS,EtOH|cpxr_EtOH_curated.gff,R1,CpxRR1_S1_R1.bw,R2,CpxRR2_S2_R2.bw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cra,Escherichia coli,K-12 MG1655,M9,"Fructose, Galactose, or Acetate",NC_000913_3,e_coli,NaN,8,4,known_TF,acetate|cra_acetate_curated.gff;M9|cra_M9_cura...,Glu – R1,cra_glu_1.bw,Glu – R2,cra_glu_2.bw,Fru – R1,cra_fru_1.bw,Fru – R2,cra_fru_2.bw,Gal – R1,cra_gal_1.bw,Gal – R2,cra_gal_2.bw,Ace – R1,cra_ace_1.bw,Ace – R2,cra_ace_2.bw
3,Fur,Escherichia coli,K-12 MG1655,M9,Fe or DPD,NC_000913_3,e_coli,NaN,4,2,known_TF,fe|fur_fe_curated.gff;dpd|fur_dpd_curated.gff,Fe – R1,fur_fe_1.bw,Fe – R2,fur_fe_2.bw,DPD – R1,fur_dpd_1.bw,DPD – R2,fur_dpd_2.bw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GadE,Escherichia coli,K-12 MG1655,M9,NaN,NC_000913_3,e_coli,NaN,2,1,known_TF,M9|gade_M9_curated.gff,R1,gade_1.bw,R2,gade_2.bw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,GadW,Escherichia coli,K-12 MG1655,M9,NaN,NC_000913_3,e_coli,NaN,2,1,known_TF,M9|gadw_M9_curated.gff,R1,gadw_1.bw,R2,gadw_2.bw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,GadX,Escherichia coli,K-12 MG1655,M9,NaN,NC_000913_3,e_coli,NaN,2,1,known_TF,M9|gadx_M9_curated.gff,R1,gadx_1.bw,R2,gadx_2.bw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,KdpE,Escherichia coli,K-12 MG1655,TMA,KCl,NC_000913_3,e_coli,NaN,2,1,TCS,KCl|kdpe_KCl_curated.gff,R1,KdpeE_R2_S34_R1.bw,R2,KdpeE_R2_S34_R2.bw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,PhoB,Escherichia coli,K-12 MG1655,M9P,NaN,NC_000913_3,e_coli,NaN,4,1,TCS,M9P|phob_M9P_curated.gff,bio-rep1 – R1,PhoB_1_S16_R1.bw,bio-rep1 – R2,PhoB_1_S16_R2.bw,bio-rep2 – R1,PhoB_2_S6_R1.bw,bio-rep2 – R2,PhoB_2_S6_R2.bw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
strain = 'NC_000913_3'
DF_gene_info = pd.read_csv(org_folder+strain+'/annotation/gene_info.csv',index_col=0)
DF_gene_info['start_codon_pos'] = [row.start if row.strand == '+' else row.stop for idx,row in DF_gene_info.iterrows()]
TRN = pd.read_csv(org_folder+strain+'/annotation/trn.csv',index_col=0)

DF_gene_info

,start,stop,strand,gene_name,length,operon,cog,start_codon_pos
b0001,189,255,+,thrL,66,thrLABC,No COG Annotation,189
b0002,336,2799,+,thrA,2463,thrLABC,No COG Annotation,336
b0003,2800,3733,+,thrB,933,thrLABC,Amino acid transport and metabolism,2800
b0004,3733,5020,+,thrC,1287,thrLABC,Amino acid transport and metabolism,3733
b0005,5233,5530,+,yaaX,297,yaaX,Function unknown,5233
...,...,...,...,...,...,...,...,...
b4399,4636695,4638120,+,creC,1425,creABCD,Signal transduction mechanisms,4636695
b4400,4638177,4639530,+,creD,1353,creABCD,Defense mechanisms,4638177
b4401,4639589,4640306,-,arcA,717,arcA,Signal transduction mechanisms,4640306
b4402,4640401,4640542,+,yjjY,141,yjjY,Function unknown,4640401


In [20]:
for TF_index in range(len(TF_list)):

    TF_name= TF_list.TF[TF_index].lower()
    strain = TF_list.genome_id[TF_index]

    #find files and format
    curated_loc = glob.glob(org_folder+strain+"/curated_input/"+TF_name+"_*.gff")
    curated_files = [i.split("/")[-1] for i in curated_loc]
    curated_cond = [i.split("_")[1] for i in curated_files]
    table_string =";".join([curated_cond[i]+"|"+curated_files[i] for i in range(len(curated_loc))])
    TF_list.loc[TF_index, 'binding tables'] = table_string
    TF_list.loc[TF_index, 'num_cond'] = len(curated_loc)
    
    #

In [21]:
TF_list.to_csv(org_folder+'TF_list.csv')

In [ ]:
for i in range (0, len(database)): 
    peak_annot_DF = annotate_peaks(TF_list[i], TF_condition[i],database[i],500,DF_gene_info).fillna('')
    final_annot_DF = validate_peak_info(peak_annot_DF,DF_gene_info)
    venn_files = Venn_data_gen(final_annot_DF)
    final_annot_DF.to_json(out_data_table+TF_list[i]+'_binding_table.json',orient='records')
    venn_files.to_json(path.join(out_data_venn, TF_list[i]+'_venn.json'),orient='records')
    binding_width_gen(TF_list[i], database[i], '../../data/e_coli/NC_000913_3/binding_widths/')
    peak_position_gen(TF_list[i], final_annot_DF, '../../data/e_coli/NC_000913_3/positions/')

In [69]:
file = 'phob_M9P_curated.gff'
df = pd.read_csv(org_folder+strain+"/curated_input/"+file,index_col=0, 
                                 delimiter='\t', header=None, 
                                 names = ['ref','condition', 'condition_name', 
                                          "binding_peak_start",'binding_peak_end', 
                                          'binding_peak_strength', 'direction', '.','ID'])
df = df.set_index(pd.Series(range(1,len(df)+1)))
df.head()

,condition,condition_name,binding_peak_start,binding_peak_end,binding_peak_strength,direction,.,ID
1,MACE,_filtered_0.95,417086,417115,1.92,+,.,SFB
2,MACE,_filtered_0.95,1198372,1198398,1.88,+,.,GSB
3,MACE,_filtered_0.95,2774093,2774126,1.82,+,.,SFB
4,MACE,_filtered_0.95,2774321,2774354,1.82,+,.,SRB
5,MACE,_filtered_0.95,3284518,3284522,1.23,+,.,GSB


In [70]:
peak_annot_DF = annotate_peaks('PhoB', 'M9P', df, 500, DF_gene_info)
final_annot_DF = validate_peak_info(peak_annot_DF,DF_gene_info)

final_annot_DF

,index,condition,binding_peak_start,binding_peak_end,binding_peak_strength,target_locus,target_genes
1,PhoB-1,phob + M9P,417086,417115,1.92,"b0399,b0400,b0397,b0398","phoB,phoR,sbcC,sbcD"
2,PhoB-2,phob + M9P,1198372,1198398,1.88,"b1139,b1137,b1138","lit,ymfD,ymfE"
3,PhoB-3,phob + M9P,2774093,2774126,1.82,,
4,PhoB-4,phob + M9P,2774321,2774354,1.82,,
5,PhoB-5,phob + M9P,3284518,3284522,1.23,"b3138,b3139,b3140,b3141","agaB,agaC,agaD,agaI"
6,PhoB-6,phob + M9P,3321253,3321291,1.69,b3173,yhbX
7,PhoB-7,phob + M9P,3321485,3321524,2.08,b3173,yhbX
8,PhoB-8,phob + M9P,3911549,3911584,8.13,"b3724,b3725,b3726,b3727,b3728","phoU,pstB,pstA,pstC,pstS"
9,PhoB-9,phob + M9P,3911629,3911670,6.93,"b3724,b3725,b3726,b3727,b3728","phoU,pstB,pstA,pstC,pstS"
10,PhoB-10,phob + M9P,4003031,4003074,2.05,"b3817,b3818","yigF,yigG"


In [60]:
len(df)

136

In [40]:
float('nan')

nan

In [47]:
[i for i in peak_annot_DF.genes_m][0]

nan

In [26]:
peak_annot_DF

,index,condition,binding_peak_start,binding_peak_end,binding_peak_strength,TU_p,genes_p,TU_m,genes_m
1,Dps-1,dps + expo,235,261,1.50,thrLABC,"b0001,b0002,b0003,b0004",NaN,NaN
2,Dps-2,dps + expo,111536,111543,2.28,,,,
3,Dps-3,dps + expo,167456,167466,1.95,fhuACDB,"b0150,b0151,b0152,b0153",NaN,NaN
4,Dps-4,dps + expo,246985,247015,1.13,yafL,b0227,dinJ-yafQ,"b0225,b0226"
5,Dps-5,dps + expo,256349,256373,1.54,"gpt,frsA","b0238,b0239",NaN,NaN
...,...,...,...,...,...,...,...,...,...
132,Dps-132,dps + expo,4406423,4406463,1.01,nsrR-rnr-rlmB-yjfIJ,"b4178,b4179,b4180,b4181,b4182",NaN,NaN
133,Dps-133,dps + expo,4474258,4474288,1.11,bdcR,b4251,bdcA,b4249
134,Dps-134,dps + expo,4527665,4527698,1.11,NaN,NaN,sgcXBCQAER,"b4300,b4301,b4302,b4303,b4304,b4565,b4305"
135,Dps-135,dps + expo,4566141,4566162,1.19,NaN,NaN,yjiML,"b4334,b4335"


In [ ]:
def annotate_peaks(TF_name, TF_condition, peak_df,margin,gene_info):
    
    res_df = peak_df.copy()
#     TF = peak_df.condition_name[1][:4]
    for i,row in res_df.iterrows():
        pos = row['binding_peak_start']
        # Identify genes within MARGIN nt of binding peak
        close_genes = gene_info[(gene_info.start_codon_pos > pos-margin) 
                                & (gene_info.start_codon_pos < pos+margin)]
        for strand,group in close_genes.groupby('strand'):
            #Remove genes that are completely transcribed before binding peak
            if strand == '+':
                group = group[group.stop > pos]
            else:
                group = group[group.start < pos]

            operon = group.operon.unique()
            # Ensure that we're only identifying one operon on either side of binding peak
            if len(operon) > 1:
                print (operon)

            # Get all genes in operon
            bnums = gene_info[gene_info.operon.isin(operon)].index
            
            ## Add gene information to dataframe
            if strand == '+':
                res_df.loc[i,'TU_p'] = ','.join(operon)
                res_df.loc[i,'genes_p'] = ','.join(bnums)
            else:
                res_df.loc[i,'TU_m'] = ','.join(operon)
                res_df.loc[i,'genes_m'] = ','.join(bnums)
    res_df['index'] = [TF_name +'-' + str(i) for i in range(1,peak_df.shape[0]+1)]
    res_df['condition'] = [ TF_name.lower() + " + " + TF_condition for i in peak_df.condition_name]
#     [peak_df.condition_name[2][:4]+' + '+peak_df.condition_name[1][5:8]]*peak_df.shape[0]
    cols = ['index','condition','binding_peak_start','binding_peak_end',
            'binding_peak_strength','TU_p','genes_p','TU_m','genes_m']
    return res_df.reindex(columns = cols)